<a href="https://colab.research.google.com/github/lehorhe/-/blob/main/korektor_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. Instalacja i Konfiguracja Środowiska
# Instalujemy najnowsze SDK Google Gen AI oraz python-docx
!pip install -q -U google-genai python-docx

import os
import sys
import time
import difflib
import datetime
from google.colab import files
from google.colab import userdata

# Nowe SDK (v1.0+) - zastępuje google.generativeai
from google import genai
from google.genai import types

# Biblioteki do obsługi Word (OpenXML)
from docx import Document
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.shared import Pt

print("Środowisko skonfigurowane pomyślnie.")

Środowisko skonfigurowane pomyślnie.


In [3]:
# @title 2. Inicjalizacja Klienta Gemini
# Pobranie klucza z sekretów. Użytkownik musi dodać klucz o nazwie 'GEMINI_API_KEY'
try:
    API_KEY = userdata.get('GEMINI_API_KEY')
except Exception as e:
    print("❌ BŁĄD: Nie znaleziono klucza API w sekretach.")
    print("Instrukcja: Kliknij ikonę klucza (po lewej), dodaj nowy sekret 'GEMINI_API_KEY' i wklej swój klucz z AI Studio.")
    raise e

# Inicjalizacja klienta
client = genai.Client(api_key=API_KEY)

# Definicja dostępnych modeli
# Gemini 3.0 Flash (Preview) - wysoka szybkość, niski koszt
MODEL_FLASH = "gemini-2.0-flash"
# Alternatywnie: "gemini-3.0-flash-preview" jeśli dostępne w danym regionie/projekcie.
# Zalecamy 2.0 Flash jako bezpieczny fallback, dopóki 3.0 nie wyjdzie z fazy ścisłego preview.

# Gemini 1.5 Pro - wysoka jakość, długi kontekst [3]
MODEL_PRO = "gemini-2.5-pro"

print(f"✅ Klient zainicjalizowany. Dostępne konfiguracje:")
print(f"   - Szybka korekta: {MODEL_FLASH}")
print(f"   - Głęboka redakcja: {MODEL_PRO}")

✅ Klient zainicjalizowany. Dostępne konfiguracje:
   - Szybka korekta: gemini-2.0-flash
   - Głęboka redakcja: gemini-2.5-pro


In [4]:
# @title 3. Definicja Silnika Track Changes (Core Logic)

def create_element(name):
    """Pomocnicza funkcja do tworzenia elementów OXML"""
    return OxmlElement(name)

def create_attribute(element, name, value):
    """Pomocnicza funkcja do dodawania atrybutów"""
    element.set(qn(name), value)

def add_tracked_change(paragraph, text, change_type, author="AI_Redaktor"):
    """
    Dodaje run do paragrafu z odpowiednim znacznikiem XML (w:ins lub w:del).
    """
    # Pobranie aktualnej daty w formacie wymaganym przez Word (ISO 8601)
    date_str = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")

    # Przypadek 1: Tekst bez zmian
    if change_type == 'equal':
        run = paragraph.add_run(text)
        return

    # Przypadek 2: Wstawienie tekstu (Insertion)
    elif change_type == 'insert':
        # Tworzymy kontener <w:ins>
        ins = create_element('w:ins')
        create_attribute(ins, 'w:id', '0') # ID powinno być unikalne, tu uproszczone
        create_attribute(ins, 'w:author', author)
        create_attribute(ins, 'w:date', date_str)

        # Wewnątrz <w:ins> musi być normalny <w:r> z <w:t>
        run = create_element('w:r')
        t = create_element('w:t')
        # Ważne: obsługa spacji w XML (xml:space="preserve")
        if text.strip() == '':
            create_attribute(t, 'xml:space', 'preserve')
        t.text = text

        run.append(t)
        ins.append(run)
        paragraph._p.append(ins)

    # Przypadek 3: Usunięcie tekstu (Deletion)
    elif change_type == 'delete':
        # Tworzymy kontener <w:del>
        del_tag = create_element('w:del')
        create_attribute(del_tag, 'w:id', '0')
        create_attribute(del_tag, 'w:author', author)
        create_attribute(del_tag, 'w:date', date_str)

        # Wewnątrz <w:del> tekst musi być w <w:delText>, a nie <w:t>!
        run = create_element('w:r')
        del_text = create_element('w:delText')
        if text.strip() == '':
            create_attribute(del_text, 'xml:space', 'preserve')
        del_text.text = text

        run.append(del_text)
        del_tag.append(run)
        paragraph._p.append(del_tag)

def apply_diff_to_paragraph(paragraph, original_text, new_text):
    """
    Kluczowa funkcja: czyści paragraf i odbudowuje go z historią zmian.
    """
    # 1. Oblicz różnice algorytmem SequenceMatcher
    matcher = difflib.SequenceMatcher(None, original_text, new_text)

    # 2. Wyczyść zawartość paragrafu (zachowując jego właściwości stylu/pPr)
    # Uwaga: paragraph.clear() usuwa runy, ale zostawia pPr (właściwości paragrafu)
    paragraph.clear()

    # 3. Iteruj po różnicach i buduj XML
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'equal':
            add_tracked_change(paragraph, original_text[i1:i2], 'equal')
        elif tag == 'delete':
            add_tracked_change(paragraph, original_text[i1:i2], 'delete')
        elif tag == 'insert':
            add_tracked_change(paragraph, new_text[j1:j2], 'insert')
        elif tag == 'replace':
            # Replace w Wordzie to zazwyczaj usunięcie starego i wstawienie nowego obok
            add_tracked_change(paragraph, original_text[i1:i2], 'delete')
            add_tracked_change(paragraph, new_text[j1:j2], 'insert')

In [11]:
# @title 4. Funkcje Przetwarzania AI

def get_ai_correction(text_block, model_name):
    """Przetwarza blok tekstu, dostosowując rygor redakcji do klasy modelu."""

    # 1. Definicja zadań specyficznych dla klasy modelu
    if "pro" in model_name.lower():
        # Zadania dla modelu PRO: Głęboka redakcja i spójność [cite: 21, 269]
        model_tasks = """- Skup się na płynności tekstu i spójności logicznej całego wywiadu.
- Koryguj błędy merytoryczne i mowę zależną (np. dopasuj płcie rozmówców do kontekstu).
- Możesz subtelnie poprawić szyk zdań, by nadać im bardziej literacki charakter."""
        temp = 0.2  # Lekka elastyczność dla lepszego stylu [cite: 316]
    else:
        # Zadania dla modelu FLASH: Szybka korekta techniczna [cite: 21, 269]
        model_tasks = """- Skup się wyłącznie na błędach ortograficznych, interpunkcyjnych i oczywistych pomyłkach zapisu.
- Nie zmieniaj struktury zdań ani stylu autora.
- Priorytetem jest wierność oryginałowi przy zachowaniu poprawności językowej."""
        temp = 0.0  # Maksymalna stabilność dla korekty technicznej

    # 2. Główna instrukcja systemowa (Core Engine)
    system_instruction = f"""Jesteś profesjonalnym, bezosobowym silnikiem redakcyjnym w wydawnictwie[cite: 205].
TWOJE ZADANIE:
{model_tasks}

ZASADY KRYTYCZNE:
1. NIE dodawaj komentarzy, powitań ani wyjaśnień (np. "Oto poprawiony tekst")[cite: 212].
2. Zwracaj DOKŁADNIE taką samą liczbę akapitów, jaką otrzymałeś.
3. Zachowaj tagi [PAR] – służą one jako jedyne separatory akapitów.
4. Jeśli fragment jest poprawny lub niejasny, zwróć go bez zmian[cite: 211].
5. Zachowaj idiolekt i styl mówiony rozmówcy (nie zmieniaj go w tekst pisany)[cite: 19, 210]."""

    try:
        response = client.models.generate_content(
            model=model_name,
            contents=text_block,
            config=types.GenerateContentConfig(
                system_instruction=system_instruction,
                temperature=temp
            )
        )
        corrected = response.text.strip()

        # Zabezpieczenie przed "gadatliwością" asystenta
        if any(bad in corrected.lower() for bad in ["przepraszam", "nie rozumiem", "proszę podać"]):
            return text_block

        return corrected
    except Exception as e:
        print(f"! Błąd API ({model_name}): {e}. Zwracam oryginał[cite: 224, 225].")
        return text_block # Fallback: brak zmian

def process_docx(input_path, output_path, model_name):
    print(f"📂 Otwieranie pliku: {input_path}")
    doc = Document(input_path)

    paragraphs = [p for p in doc.paragraphs if len(p.text.strip()) > 2] # Ignoruj puste
    total = len(paragraphs)
    print(f"📝 Znaleziono {total} akapitów do analizy.")

    for i, para in enumerate(paragraphs):
        original = para.text

        # Logowanie co 5 akapitów
        if i % 5 == 0:
            print(f"   Przetwarzanie: {i}/{total} ({(i/total)*100:.1f}%)")

        # Wywołanie AI
        corrected = get_ai_correction(original, model_name)

        # Aplikacja zmian tylko jeśli są różnice
        if corrected!= original:
            apply_diff_to_paragraph(para, original, corrected)

        # Rate Limiting (ważne dla darmowego API)
        time.sleep(1.5)

    print(f"💾 Zapisywanie pliku: {output_path}")
    doc.save(output_path)
    print("✅ Gotowe!")

In [17]:
# @title 5. URUCHOM PROCES
# Wybierz model z listy rozwijanej
MODEL = "gemini-2.5-pro" # @param ["gemini-2.5-pro", "gemini-2.0-flash"]

print("⬆️ Prześlij plik.docx do redakcji:")
uploaded = files.upload()

if uploaded:
    input_file = next(iter(uploaded))
    output_file = f"redakcja_{input_file}"

    process_docx(input_file, output_file, MODEL)

    print("⬇️ Pobieranie wyniku...")
    files.download(output_file)
else:
    print("Nie przesłano pliku.")

⬆️ Prześlij plik.docx do redakcji:


Saving redakcja_redakcja_0.ZOOM0004p_pierwsze nagranie (6) (1).docx to redakcja_redakcja_0.ZOOM0004p_pierwsze nagranie (6) (1) (1).docx
📂 Otwieranie pliku: redakcja_redakcja_0.ZOOM0004p_pierwsze nagranie (6) (1) (1).docx
📝 Znaleziono 181 akapitów do analizy.
   Przetwarzanie: 0/181 (0.0%)
   Przetwarzanie: 5/181 (2.8%)
   Przetwarzanie: 10/181 (5.5%)
   Przetwarzanie: 15/181 (8.3%)
   Przetwarzanie: 20/181 (11.0%)
   Przetwarzanie: 25/181 (13.8%)
   Przetwarzanie: 30/181 (16.6%)
   Przetwarzanie: 35/181 (19.3%)
   Przetwarzanie: 40/181 (22.1%)
   Przetwarzanie: 45/181 (24.9%)
   Przetwarzanie: 50/181 (27.6%)
   Przetwarzanie: 55/181 (30.4%)
   Przetwarzanie: 60/181 (33.1%)
   Przetwarzanie: 65/181 (35.9%)
   Przetwarzanie: 70/181 (38.7%)
   Przetwarzanie: 75/181 (41.4%)
   Przetwarzanie: 80/181 (44.2%)
   Przetwarzanie: 85/181 (47.0%)
   Przetwarzanie: 90/181 (49.7%)
   Przetwarzanie: 95/181 (52.5%)
   Przetwarzanie: 100/181 (55.2%)
   Przetwarzanie: 105/181 (58.0%)
   Przetwarzanie: 1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>